In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [0]:
# Import the dataset
from keras.datasets import mnist

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [4]:
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [5]:
# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


In [6]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 129us/step - loss: 0.4484 - acc: 0.8684 - val_loss: 0.2012 - val_acc: 0.9402
Epoch 2/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.2176 - acc: 0.9358 - val_loss: 0.1411 - val_acc: 0.9563
Epoch 3/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.1632 - acc: 0.9515 - val_loss: 0.1249 - val_acc: 0.9624
Epoch 4/10
60000/60000 [==============================] - 4s 70us/step - loss: 0.1368 - acc: 0.9589 - val_loss: 0.1054 - val_acc: 0.9695
Epoch 5/10
60000/60000 [==============================] - 4s 71us/step - loss: 0.1203 - acc: 0.9646 - val_loss: 0.1002 - val_acc: 0.9703
Epoch 6/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.1076 - acc: 0.9671 - val_loss: 0.0915 - val_acc: 0.9727
Epoch 7/10
60000/60000 [==============================] - 4s 65us/step - loss: 0.0965 - acc: 0.9702 - val_loss: 0.0921 - val_ac

In [8]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 16s 274us/step - loss: 0.2675 - acc: 0.9171 - val_loss: 0.0573 - val_acc: 0.9823
Epoch 2/10
60000/60000 [==============================] - 12s 201us/step - loss: 0.0885 - acc: 0.9732 - val_loss: 0.0433 - val_acc: 0.9850
Epoch 3/10
60000/60000 [==============================] - 12s 195us/step - loss: 0.0650 - acc: 0.9814 - val_loss: 0.0392 - val_acc: 0.9876
Epoch 4/10
60000/60000 [==============================] - 12s 195us/step - loss: 0.0553 - acc: 0.9836 - val_loss: 0.0335 - val_acc: 0.9892
Epoch 5/10
60000/60000 [==============================] - 12s 197us/step - loss: 0.0481 - acc: 0.9857 - val_loss: 0.0308 - val_acc: 0.9895
Epoch 6/10
60000/60000 [==============================] - 12s 195us/step - loss: 0.0453 - acc: 0.9862 - val_loss: 0.0330 - val_acc: 0.9886
Epoch 7/10
60000/60000 [===================

In [9]:
# Training parameters.
batch_size = 64
num_classes = 10
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 246s 4ms/step - loss: 0.8989 - acc: 0.6976 - val_loss: 0.5309 - val_acc: 0.8234
Epoch 2/3
60000/60000 [==============================] - 262s 4ms/step - loss: 0.3395 - acc: 0.8947 - val_loss: 0.3157 - val_acc: 0.8916
Epoch 3/3
60000/60000 [==============================] - 199s 3ms/step - loss: 0.2195 - acc: 0.9326 - val_loss: 0.1993 - val_acc: 0.9374
Test loss: 0.19926961847543717
Test accuracy: 0.9374


In [11]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=14,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/14
60000/60000 [==============================] - 129s 2ms/step - loss: 0.0484 - acc: 0.9850 - val_loss: 0.0770 - val_acc: 0.9768
Epoch 2/14
60000/60000 [==============================] - 130s 2ms/step - loss: 0.0461 - acc: 0.9855 - val_loss: 0.0422 - val_acc: 0.9865
Epoch 3/14
60000/60000 [==============================] - 130s 2ms/step - loss: 0.0437 - acc: 0.9860 - val_loss: 0.0530 - val_acc: 0.9847
Epoch 4/14
60000/60000 [==============================] - 129s 2ms/step - loss: 0.0417 - acc: 0.9869 - val_loss: 0.0551 - val_acc: 0.9827
Epoch 5/14
60000/60000 [==============================] - 138s 2ms/step - loss: 0.0407 - acc: 0.9873 - val_loss: 0.0466 - val_acc: 0.9859
Epoch 6/14
60000/60000 [==============================] - 216s 4ms/step - loss: 0.0400 - acc: 0.9876 - val_loss: 0.0505 - val_acc: 0.9853
Epoch 7/14
60000/60000 [==============================] - 288s 5ms/step - loss: 0.0364 - acc: 0.9888 - val_loss: 0.0661 - 